In [36]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

import string  
import nltk
import re
import scipy
import matplotlib.pyplot as plt
import urllib 
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error

train = pd.read_csv('train.csv')
display(train)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [37]:
train['keyword'].fillna('no keyword', inplace = True) 
train['keyword'] = train['keyword'].str.replace('%20', ' ')
train['location'].fillna('no location', inplace = True)

train['length'] = train['text'].str.len()

train.head()

,id,keyword,location,text,target,length
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88


In [38]:
#limpiando el texto

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train['clean_text'] = train['text'].str.lower()

train['clean_text'] = train['clean_text'].apply(only_letters)    

#Tokenización

train['clean_text'] = train['clean_text'].apply(word_tokenize)

#Remove stopwords

train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

#Lemmatization                                                                       

train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

train['clean_text'] = train['clean_text'].apply(transform_to_text)
display(train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...,...
7608,10869,no keyword,no location,Two giant cranes holding a bridge collapse int...,1,83,two giant crane holding bridge collapse nearby...
7609,10870,no keyword,no location,@aria_ahrary @TheTawniest The out of control w...,1,125,ariaahrary thetawniest control wild fire calif...
7610,10871,no keyword,no location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,utckm volcano hawaii
7611,10872,no keyword,no location,Police investigating after an e-bike collided ...,1,137,police investigating ebike collided car little...


In [39]:
#agregando algunos features

keywords_list = list(set(train.keyword))  #todas las keywords únicas

locations_list = list(set(train.location)) #todas las locaciones únicas
locations_list.remove('no location')

def amount_keywords(tweet):
    cont = 0
    for keyword in keywords_list:
        if keyword in tweet:
            cont += 1
    return cont

train['amount_words'] = train['clean_text'].str.split(' ').apply(lambda x: len(x))  #conviene que cuente del limpio o del original??

train['amount_keywords_in_tweet'] = train['clean_text'].apply(lambda x: amount_keywords(x))


tiene_mas_de_un_kw = train[train['amount_keywords_in_tweet'] > 1]

tiene_un_solo_kw = train[train['amount_keywords_in_tweet'] == 1]

no_tiene_kw = train[train['amount_keywords_in_tweet'] == 0]

porcentaje_target_mas_de_un_kw = tiene_mas_de_un_kw['target'].mean()
print("porcentaje target de tweets con mas de un kw", porcentaje_target_mas_de_un_kw)

porcentaje_target_un_solo_kw = tiene_un_solo_kw['target'].mean()
print("porcentaje target de tweets con un solo kw", porcentaje_target_un_solo_kw)

porcentaje_target_sin_kw = no_tiene_kw['target'].mean()
print("porcentaje target de tweets sin kw", porcentaje_target_sin_kw)

print("el maximo de cant de keywords en un tweet es", tiene_mas_de_un_kw['amount_keywords_in_tweet'].max())

def amount_locations(tweet):    #esta función podría unificarse con la de keywords pasandole por parámetro la lista a recorrer
    cont = 0
    for location in locations_list:
        if location in tweet:
            cont += 1
    return cont

train['amount_locations_in_tweet'] = train['clean_text'].apply(lambda x: amount_locations(x))

mas_de_1_location = train[train['amount_locations_in_tweet'] > 1]

un_solo_location = train[train['amount_locations_in_tweet'] == 1]

no_tiene_location = train[train['amount_locations_in_tweet'] == 0]

porcentaje_target_mas_de_un_location = mas_de_1_location['target'].mean()
print("porcentaje target de tweets con mas de una locacion", porcentaje_target_mas_de_un_location)

porcentaje_target_un_solo_location = un_solo_location['target'].mean()
print("porcentaje target de tweets con una sola locacion", porcentaje_target_un_solo_location)

porcentaje_target_sin_location = no_tiene_location['target'].mean()
print("porcentaje target de tweets sin locacion", porcentaje_target_sin_location)

print("el maximo de cant de locaciones en un tweet es", mas_de_1_location['amount_locations_in_tweet'].max())

porcentaje target de tweets con mas de un kw 0.606816584680253
porcentaje target de tweets con un solo kw 0.3446327683615819
porcentaje target de tweets sin kw 0.2025862068965517
el maximo de cant de keywords en un tweet es 7
porcentaje target de tweets con mas de una locacion 0.504933504933505
porcentaje target de tweets con una sola locacion 0.4489409984871407
porcentaje target de tweets sin locacion 0.34382107657316147
el maximo de cant de locaciones en un tweet es 8


In [40]:
mas_de_un_location_y_mas_de_un_keyword = train[(train['amount_locations_in_tweet'] > 1) & (train['amount_keywords_in_tweet'] > 1)]
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword = mas_de_un_location_y_mas_de_un_keyword.target.mean()
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword

0.6990196078431372

In [41]:
#construyendo los datos de entrenamiento y de test
selected_df = train[['length','amount_words','amount_keywords_in_tweet','amount_locations_in_tweet']] 
#ver después de codificar las variables categóricas para agregarlas
X, y = selected_df,train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [42]:
#Random forest por defecto
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
rf_model.feature_importances_

RMSE: 0.510235


array([0.47077955, 0.22850198, 0.14711824, 0.15360023])

In [43]:
#Random forest prueba2
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 100, max_features = 4, min_samples_split = 3, max_depth = 3)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.462713


In [44]:
#Random forest prueba3
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 5, max_depth = 4)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.459952


In [45]:
#Random forest prueba4
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 5, max_depth = 6,
                                min_samples_leaf = 4)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.457056


In [46]:
#Random forest prueba5
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 8, max_depth = 6,
                                min_samples_leaf = 10)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.456772


In [47]:
#Random forest prueba6
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 10, max_depth = 6,
                                min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.456633


In [48]:
#Random forest prueba7
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 10, max_depth = 6,
                                min_samples_leaf = 20)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.456374


In [49]:
#Random forest prueba8
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 10, max_depth = 7,
                                min_samples_leaf = 25)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.455718


In [50]:
#Random forest prueba9
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 10, max_depth = 8,
                                min_samples_leaf = 25)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.455281


In [51]:
#Random forest prueba10
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 10, max_depth = 10,
                                min_samples_leaf = 25)
                                        #VARIANDO MIN_SAMPLES_SPLIT NO CAMBIA NADA
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.455149


In [52]:
#Random forest prueba11
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 10, max_depth = 11,
                                min_samples_leaf = 25)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.455133


In [53]:
#Random forest prueba12
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 206, max_features = 2, min_samples_split = 10, max_depth = 11,
                                min_samples_leaf = 25)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.455130


In [54]:
#Random forest prueba13
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 205, max_features = 2, min_samples_split = 10, max_depth = 11,
                                min_samples_leaf = 25)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.455124


In [55]:
#Random forest prueba14
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 205, max_features = 2, min_samples_split = 10, max_depth = 11,
                                min_samples_leaf = 26)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.455008


In [56]:
#Random forest prueba15
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 205, max_features = 2, min_samples_split = 10, max_depth = 11,
                                min_samples_leaf = 26)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.455008
